In [47]:
import pickle
import pandas as pd
from sklearn.preprocessing import StandardScaler
from datetime import date, datetime, timedelta

In [48]:
start_date = datetime(2022, 4, 1)
dates = [start_date + timedelta(days=i) for i in range(365 * 3)]

df = pd.DataFrame({'Date': dates})

df['day_of_year'] = df['Date'].dt.dayofyear
df['month'] = df['Date'].dt.month
df['day_of_week'] = df['Date'].dt.dayofweek  # Monday=0, Sunday=6

scaler = StandardScaler()
scaled_features = scaler.fit_transform(df[['day_of_year', 'month', 'day_of_week']])

df_scaled = pd.DataFrame(scaled_features, columns=['day_of_year_scaled', 'month_scaled', 'day_of_week_scaled'])
df = pd.concat([df, df_scaled], axis=1)

df

,Date,day_of_year,month,day_of_week,day_of_year_scaled,month_scaled,day_of_week_scaled
0,2022-04-01,91,4,4,-0.874648,-0.732147,0.497149
1,2022-04-02,92,4,5,-0.865167,-0.732147,0.997036
2,2022-04-03,93,4,6,-0.855686,-0.732147,1.496924
3,2022-04-04,94,4,0,-0.846205,-0.732147,-1.502402
4,2022-04-05,95,4,1,-0.836724,-0.732147,-1.002515
...,...,...,...,...,...,...,...
1090,2025-03-26,85,3,2,-0.931535,-1.022093,-0.502627
1091,2025-03-27,86,3,3,-0.922054,-1.022093,-0.002739
1092,2025-03-28,87,3,4,-0.912573,-1.022093,0.497149
1093,2025-03-29,88,3,5,-0.903091,-1.022093,0.997036


In [49]:
# Opening saved model
with open("model/classifier.pkl", "rb") as file:
    classifier = pickle.load(file)


In [50]:
product_data = pd.read_csv("dataset/Project Dataset/product.csv")

In [51]:
product_data = product_data.sort_values("product").reset_index().drop(columns=['index'])
product_data

,product,id,price
0,12 C AC,101,67300
1,12 inch Fan,57,3990
2,14 inch Fan,56,4490
3,17 inch Fan,54,6290
4,18 A AC,99,49900
...,...,...,...
118,Tornedo Fan,66,1090
119,Wall Move,68,3130
120,Washing Machine,84,19900
121,Water Filter,69,3900


In [52]:
today = date.today()
prediction_dates = [today + timedelta(days = i) for i in range(30)]

sql = "INSERT INTO `predictions` (`id`, `date`, `product_id`, `unit`, `user_id`, `created_at`, `updated_at`) VALUES "

for date in prediction_dates:
    result = classifier.predict(pd.DataFrame({'day_of_year': date.timetuple().tm_yday, 'month':date.month, 'day_of_week':date.weekday()}, index = [0]))
    i = 0
    for value in result[0]:
        if not value == 0:
            sql += "(NULL, '" + str(date) + "', '" + str(product_data['id'][i]) + "', '" + str(value) + "', '5', CURRENT_TIMESTAMP, CURRENT_TIMESTAMP),"
        i += 1
print(sql)

INSERT INTO `predictions` (`id`, `date`, `product_id`, `unit`, `user_id`, `created_at`, `updated_at`) VALUES (NULL, '2024-09-07', '54', '2', '5', CURRENT_TIMESTAMP, CURRENT_TIMESTAMP),(NULL, '2024-09-07', '141', '1', '5', CURRENT_TIMESTAMP, CURRENT_TIMESTAMP),(NULL, '2024-09-07', '127', '1', '5', CURRENT_TIMESTAMP, CURRENT_TIMESTAMP),(NULL, '2024-09-07', '88', '1', '5', CURRENT_TIMESTAMP, CURRENT_TIMESTAMP),(NULL, '2024-09-07', '83', '1', '5', CURRENT_TIMESTAMP, CURRENT_TIMESTAMP),(NULL, '2024-09-07', '81', '1', '5', CURRENT_TIMESTAMP, CURRENT_TIMESTAMP),(NULL, '2024-09-07', '67', '1', '5', CURRENT_TIMESTAMP, CURRENT_TIMESTAMP),(NULL, '2024-09-08', '56', '1', '5', CURRENT_TIMESTAMP, CURRENT_TIMESTAMP),(NULL, '2024-09-08', '153', '1', '5', CURRENT_TIMESTAMP, CURRENT_TIMESTAMP),(NULL, '2024-09-08', '87', '1', '5', CURRENT_TIMESTAMP, CURRENT_TIMESTAMP),(NULL, '2024-09-08', '133', '1', '5', CURRENT_TIMESTAMP, CURRENT_TIMESTAMP),(NULL, '2024-09-08', '88', '1', '5', CURRENT_TIMESTAMP, CURREN